In [251]:
import sys
import os
import pandas as pd
from ema_workbench.analysis.parcoords import ParallelAxes, get_limits
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 5]
sys.path.append('..')
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler #to rescale Euclidean mean
from itertools import chain
from platypus import Solution, Problem
import pickle

import problem_formulation_original, problem_formulation_gini, problem_formulation_euclidean

# Loading solutions (only objectives)

## Make a dictionary out of all solutions

In [252]:
#Add your directory here
# os.chdir('/Users/farleyrimon/Documents/Model_results/Experiment 2')

pareto_sets = {}
# pareto_sets_test = {}

ethical_formulations = ['TraditionalPrinciple',
                        'CombinedTraditionalGiniMean', 
                        'CombinedTraditionalGiniStd',
                        'CombinedTraditionalGiniRatioStdMean',
                        'CombinedTraditionalEuclideanMean',
                        'CombinedTraditionalEuclideanStd',
                        'CombinedTraditionalEuclideanRatioStdMean',
]


for entry in ethical_formulations:
    solutions = []
    folder_destination = '/Users/farleyrimon/Documents/GitHub/MUSEH2O/susquehanna/output_farley'
#     output = 'output_farley'
#     name = entry.__name__
    output_dir = f"{folder_destination}/{entry}/"
    for filename in os.listdir(output_dir):
        if filename.endswith('solution.csv'):
            df_temp = pd.read_csv(f"{output_dir}{filename}", header=0)
            print(f"\n {entry} \n {df_temp.describe()}")
            solutions.append(df_temp.values.tolist())
#             pareto_sets_test[name] = list(chain.from_iterable(solutions))
            pareto_sets[entry] = df_temp
# df_temp['equity'] *= -1
pareto_sets['TraditionalPrinciple']['equity'] = 0


 TraditionalPrinciple 
        hydropower revenue  atomic plant reliability  baltimore reliability  \
count         1581.000000               1581.000000            1581.000000   
mean            61.930592                  0.778554               0.543251   
std              8.612459                  0.193601               0.212062   
min             31.033128                  0.043447               0.004994   
25%             56.505769                  0.659250               0.369220   
50%             58.762988                  0.837173               0.550819   
75%             68.559193                  0.948190               0.715390   
max             80.984192                  0.999712               0.912923   

       chester reliability  environment shortage  recreation reliability  
count          1581.000000           1581.000000             1581.000000  
mean              0.778777              0.068371                0.995798  
std               0.155622              0.01620


 CombinedTraditionalGiniStd 
        hydropower revenue  atomic plant reliability  baltimore reliability  \
count         1680.000000               1680.000000            1680.000000   
mean            64.256155                  0.739675               0.507694   
std             10.480794                  0.215808               0.201179   
min             32.895519                  0.031485               0.003299   
25%             56.710473                  0.590566               0.365189   
50%             59.667427                  0.796682               0.509758   
75%             75.092049                  0.920864               0.655782   
max             81.176646                  0.999788               0.921039   

       chester reliability  environment shortage  recreation reliability  \
count          1680.000000           1680.000000             1680.000000   
mean              0.676261              0.068693                0.995259   
std               0.199913            


 CombinedTraditionalEuclideanMean 
        hydropower revenue  atomic plant reliability  baltimore reliability  \
count         2216.000000               2216.000000            2216.000000   
mean            52.332433                  0.705913               0.621621   
std             13.163635                  0.181447               0.180244   
min             13.677113                  0.106084               0.061181   
25%             46.904917                  0.634879               0.507250   
50%             52.111406                  0.732855               0.662419   
75%             58.357271                  0.832896               0.759855   
max             80.143285                  0.999371               0.915380   

       chester reliability  environment shortage  recreation reliability  \
count          2216.000000           2216.000000             2216.000000   
mean              0.662470              0.164802                0.899691   
std               0.167681      

# Generate reference sets

We ought to make sure that the reference set is separate for every principle.

In [253]:
pareto_sets_w_variables = {}

for entry in ethical_formulations:
    solutions = []
    sets_per_seed = []
    name = entry
    output_dir = f"{folder_destination}/{name}/"
    
    seeds = [10,20,30,40,50]
    for seed in seeds:
        solutions = pd.read_csv(os.path.join(output_dir, f"{seed}_solution.csv"))
        variables = pd.read_csv(os.path.join(output_dir, f"{seed}_variables.csv"), header=None)
        
        combined = pd.concat([variables, solutions], axis=1)
        sets_per_seed.append(combined)
    
    pareto_sets_w_variables[entry] = pd.concat(sets_per_seed)


print("Amount of solutions for each principle:")
for problem_form in pareto_sets_w_variables:
    print(problem_form, len(pareto_sets_w_variables[problem_form]))
    

Amount of solutions for each principle:
TraditionalPrinciple 7779
CombinedTraditionalGiniMean 8514
CombinedTraditionalGiniStd 7997
CombinedTraditionalGiniRatioStdMean 5582
CombinedTraditionalEuclideanMean 12611
CombinedTraditionalEuclideanStd 9013
CombinedTraditionalEuclideanRatioStdMean 6233


The range of solutions found is wide. From 5000 to 12500, almost 3x more. It is important to play with the epsilons when calculating the reference set to avoid having either too many solutions (as above) or too less solutions (can happen with a too large epsilon)

In [254]:
# quickly add an extra column of Equity for the traditional principle
pareto_sets_w_variables['TraditionalPrinciple']['equity'] = 0


# indicate you want data stored in project folder
folder_destination_data = '/Users/farleyrimon/Documents/GitHub/MUSEH2O/susquehanna/refsets'

# To determine convergence we need to have the range of the global reference set in equal range across objectives. For the distributive justice formulations this is a problem because each formulation is between its own range. What we do is we create a reference set, solely for the convergence calculations. We repeat this part to calculate the reference set for the logging of objectives

In [255]:
def normalizer(df):
    # Access the euclidean mean so we normalize it between the normal ranges of each coefficient.
    dataframe = df.copy()
    equity_column = dataframe['equity']
    
    equity_values = equity_column.values.reshape(-1, 1)  # Reshape the column to a 2D array for normalization
    scaler = MinMaxScaler()
    
    # Normalize the column between 0 and 1
    normalized_equity = scaler.fit_transform(equity_values)
    
    # Update the 'Equity' column with the normalized values
    dataframe['equity'] = normalized_equity
    
    return dataframe

def column_renamer(df):
    new_columns = ['hydropower revenue', 'atomic plant reliability', 'baltimore reliability', 'chester reliability', 'environment shortage', 'recreation reliability', 'equity']
    # Get the current column names
    current_columns = df.columns

    # Create a mapping dictionary to map old column names to new column names
    mapping = dict(zip(current_columns[-7:], new_columns))

    # Rename the columns using the mapping dictionary
    df = df.rename(columns=mapping)
    return df

In [256]:
# this code produces the reference set per principle, but in the stored results also included the associated decision variables
import pareto

if not os.path.exists(folder_destination_data):
    os.mkdir(folder_destination_data)
    print("Folder created:", folder_destination_data)
else:
    print("Folder already exists:", folder_destination_data)
    
reference_sets = {}

for problem_formulation in pareto_sets_w_variables:
    data = pareto_sets_w_variables[problem_formulation]
    
    if problem_formulation == 'TraditionalPrinciple':
        nondominated = pareto.eps_sort([data.values], [32, 33, 34, 35, 36, 37], [0.5,0.1,0.1,0.1,0.01,0.1,], maximize=[32, 33, 34, 35, 37])
        reference_sets[problem_formulation] = pd.DataFrame(nondominated)
        reference_sets[problem_formulation] = column_renamer(reference_sets[problem_formulation])

        df_nondom = pd.DataFrame(nondominated, columns=data.columns)
        df_nondom = column_renamer(df_nondom)
        df_nondom.to_csv(f"{folder_destination_data}/{problem_formulation}_refset_with_variables_convergence_format.csv", index=False, header=True)
    else:
        nondominated = pareto.eps_sort([data.values], [32, 33, 34, 35, 36, 37, 38], [0.5,0.1,0.1,0.1,0.01,0.1,0.01], maximize=[32, 33, 34, 35, 37])

        reference_sets[problem_formulation] = pd.DataFrame(nondominated)
        reference_sets[problem_formulation] = column_renamer(reference_sets[problem_formulation])

        df_nondom = pd.DataFrame(nondominated, columns=data.columns)
        df_nondom = column_renamer(df_nondom)
        df_nondom = normalizer(df_nondom)
        df_nondom.to_csv(f"{folder_destination_data}/{problem_formulation}_refset_with_variables_convergence_format.csv", index=False, header=True)
        

print("Amount of solutions in reference set for each principle:")
for problem_form in reference_sets:
    print(problem_form, len(reference_sets[problem_form]))
    

Folder already exists: /Users/farleyrimon/Documents/GitHub/MUSEH2O/susquehanna/refsets
Amount of solutions in reference set for each principle:
TraditionalPrinciple 210
CombinedTraditionalGiniMean 347
CombinedTraditionalGiniStd 256
CombinedTraditionalGiniRatioStdMean 379
CombinedTraditionalEuclideanMean 1122
CombinedTraditionalEuclideanStd 657
CombinedTraditionalEuclideanRatioStdMean 437


In [257]:
import pareto 

reference_sets_sol = {}

for problem_formulation in pareto_sets:
    data = pareto_sets[problem_formulation]
    
    if problem_formulation == 'TraditionalPrinciple':
        nondominated = pareto.eps_sort([data.values], [0,1,2,3,4,5], [0.5,0.1,0.1,0.1,0.01,0.1,], maximize=[0,1,2,3,5])
        reference_sets_sol[problem_formulation] = nondominated
        df_nondom = pd.DataFrame(nondominated, columns=['hydropower', 'atomicpowerplant', 'baltimore', 'chester', 'environment', 'recreation','equity'])
        df_nondom = column_renamer(df_nondom)
#         df_nondom = normalizer(df_nondom)
        df_nondom.to_csv(f"{folder_destination_data}/{problem_formulation}_refset_convergence_format.csv", index=False, header=True)
    else:
        nondominated = pareto.eps_sort([data.values], [0,1,2,3,4,5,6], [0.5,0.1,0.1,0.1,0.01,0.1,0.01], maximize=[0,1,2,3,5])
        reference_sets_sol[problem_formulation] = nondominated
        df_nondom = pd.DataFrame(nondominated, columns=['hydropower', 'atomicpowerplant', 'baltimore', 'chester', 'environment', 'recreation', 'equity'])
        df_nondom = column_renamer(df_nondom)
        df_nondom = normalizer(df_nondom)
        df_nondom.to_csv(f"{folder_destination_data}/{problem_formulation}_refset_convergence_format.csv", index=False, header=True)

print("Amount of solutions in reference set for each principle:")
for problem_form in reference_sets_sol:
    print(problem_form, len(reference_sets_sol[problem_form]))
    

Amount of solutions in reference set for each principle:
TraditionalPrinciple 192
CombinedTraditionalGiniMean 276
CombinedTraditionalGiniStd 286
CombinedTraditionalGiniRatioStdMean 200
CombinedTraditionalEuclideanMean 656
CombinedTraditionalEuclideanStd 384
CombinedTraditionalEuclideanRatioStdMean 307


### Global Ref Set (traditional formulation)

In [258]:
x = 0
for ethical_formulation in ethical_formulations[0:1]:
    x += len(pareto_sets[ethical_formulation])
    print(ethical_formulation, len(pareto_sets[ethical_formulation]))
print("total:", x)
    
pareto_set = {}
sollist = []
solutions = []
for entry in ethical_formulations[0:1]:
    name = entry
    output_dir = f"{folder_destination}/{name}/"
    for filename in os.listdir(output_dir):
        if filename.endswith('solution.csv'):
            sollist.append(filename[:-4])
            df_temp = pd.read_csv(f"{output_dir}{filename}", header=0)
            solutions.append(df_temp.values.tolist())
pareto_set = list(chain.from_iterable(solutions))
len(pareto_set)

nondominated = pareto.eps_sort([pareto_set], [0,1,2,3,4,5], [0.5,0.1,0.1,0.1,0.01,0.1,], maximize=[0,1,2,3,5])
df_nondom = pd.DataFrame(nondominated, columns=['hydropower revenue', 'atomic power plant reliability', 'baltimore reliability', 'chester reliability', 'environment shortage', 'recreation reliability'])
print(len(nondominated))
df_nondom.to_csv(f"{folder_destination_data}/global_refset_traditional_formulation_convergence_format.csv", index=False, header=True)

TraditionalPrinciple 1454
total: 1454
210


### Global ref set other formulations with distributive justice principles

In [259]:
x = 0
for ethical_formulation in ethical_formulations[1:]:
    x += len(pareto_sets[ethical_formulation])
    print(ethical_formulation, len(pareto_sets[ethical_formulation]))
print("total:", x)
    
pareto_set = {}
sollist = []
solutions = []
for entry in ethical_formulations[1:]:
    name = entry
    output_dir = f"{folder_destination}/{name}/"
    for filename in os.listdir(output_dir):
        if filename.endswith('solution.csv'):
            sollist.append(filename[:-4])
            df_temp = pd.read_csv(f"{output_dir}{filename}", header=0)
            solutions.append(df_temp.values.tolist())
pareto_set = list(chain.from_iterable(solutions))
len(pareto_set)

nondominated = pareto.eps_sort([pareto_set], [0,1,2,3,4,5,6], [0.5,0.1,0.1,0.1,0.01,0.1,0.01], maximize=[0,1,2,3,5])
df_nondom = pd.DataFrame(nondominated, columns=['hydropower revenue', 'atomic power plant reliability', 'baltimore reliability', 'chester reliability', 'environment shortage', 'recreation reliability', 'equity'])
print(len(nondominated))
df_nondom.to_csv(f"{folder_destination_data}/global_refset_combined_formulations_convergence_format.csv", index=False, header=True)


CombinedTraditionalGiniMean 1666
CombinedTraditionalGiniStd 1582
CombinedTraditionalGiniRatioStdMean 822
CombinedTraditionalEuclideanMean 2216
CombinedTraditionalEuclideanStd 1475
CombinedTraditionalEuclideanRatioStdMean 1161
total: 8922
667


## Global reference set across all formulations, to calculate hypervolume

In [260]:
x = 0
for ethical_formulation in ethical_formulations:
    x += len(pareto_sets[ethical_formulation])
    print(ethical_formulation, len(pareto_sets[ethical_formulation]))
print("total:", x)
    
pareto_set = {}
sollist = []
solutions = []
for entry in ethical_formulations:
    name = entry
    output_dir = f"{folder_destination}/{name}/"
    for filename in os.listdir(output_dir):
        if filename.endswith('solution.csv'):
            sollist.append(filename[:-4])
            df_temp = pd.read_csv(f"{output_dir}{filename}", header=0)
#             if entry != ethical_formulations[0]:
#                 df_temp = df_temp.iloc[:,:-1]
            solutions.append(df_temp.values.tolist())
pareto_set = list(chain.from_iterable(solutions))
len(pareto_set)

nondominated = pareto.eps_sort([pareto_set], [0,1,2,3,4,5], [0.5,0.1,0.1,0.1,0.01,0.1], maximize=[0,1,2,3,5])
df_nondom = pd.DataFrame(nondominated, columns=['hydropower revenue', 'atomic power plant reliability', 'baltimore reliability', 'chester reliability', 'environment shortage', 'recreation reliability'])
df_nondom.to_csv(f"{folder_destination_data}/global_refset_all_formulation_hv_format.csv", index=False, header=True)


TraditionalPrinciple 1454
CombinedTraditionalGiniMean 1666
CombinedTraditionalGiniStd 1582
CombinedTraditionalGiniRatioStdMean 822
CombinedTraditionalEuclideanMean 2216
CombinedTraditionalEuclideanStd 1475
CombinedTraditionalEuclideanRatioStdMean 1161
total: 10376


ValueError: 6 columns passed, passed data had 7 columns

# Because of this we also need to convert the column of equity in the Hypervolume file to normalized format!

In [ ]:
def normalizer_hv(df):
    # Access the euclidean mean so we normalize it between the normal ranges of each coefficient.
    dataframe = df.copy()
    equity_column = dataframe['6']  # Update column access here
    
    equity_values = equity_column.values.reshape(-1, 1)  # Reshape the column to a 2D array for normalization
    scaler = MinMaxScaler()
    
    # Normalize the column between 0 and 1
    normalized_equity = scaler.fit_transform(equity_values)
    
    # Update the 'Equity' column with the normalized values
    dataframe['6'] = normalized_equity  # Update column assignment here
    
    return dataframe

In [ ]:
#Add your directory here
# os.chdir('/Users/farleyrimon/Documents/Model_results/Experiment 2')

hv_sets = {}
# pareto_sets_test = {}
# seeds = np.arange(10, 51, 10).tolist()

for entry in ethical_formulations:
    solutions = []
    folder_destination = '/Users/farleyrimon/Documents/GitHub/MUSEH2O/susquehanna/output_farley'
#     output = 'output_farley'
#     name = entry.__name__
    output_dir = f"{folder_destination}/{entry}/"
    if entry != ethical_formulations[0]:
        print(output_dir)

        for filename in os.listdir(output_dir):
            if filename.endswith('hypervolume.csv'):
                print(filename)
                df_temp = pd.read_csv(f"{output_dir}{filename}", header=0)
#                 print(df_temp.columns)
#                 df_temp = column_renamer(df_temp)
                df_temp = normalizer_hv(df_temp)
                df_temp = df_temp.set_index('Unnamed: 0')
                df_temp.to_csv(f"{folder_destination}/{entry}/{filename.split('.')[0]}_normalized_equity.csv", index=True, header=True)
                solutions.append(df_temp.values.tolist())
                hv_sets[entry] = df_temp


# The normal reference set calculation (without normalizing), which is needed to run the log of objectives.

In [261]:
# this code produces the reference set per principle, but in the stored results also included the associated decision variables
import pareto

if not os.path.exists(folder_destination_data):
    os.mkdir(folder_destination_data)
    print("Folder created:", folder_destination_data)
else:
    print("Folder already exists:", folder_destination_data)
    
reference_sets = {}
reference_sets_sol = {}


for problem_formulation in pareto_sets_w_variables:
    data = pareto_sets_w_variables[problem_formulation]
    
    if problem_formulation == 'TraditionalPrinciple':
        nondominated = pareto.eps_sort([data.values], [32, 33, 34, 35, 36, 37], [0.5,0.1,0.1,0.1,0.01,0.1], maximize=[32, 33, 34, 35, 37])
        reference_sets[problem_formulation] = pd.DataFrame(nondominated)
        reference_sets[problem_formulation] = column_renamer(reference_sets[problem_formulation])
        df_nondom = pd.DataFrame(nondominated, columns=data.columns)
        df_nondom.to_csv(f"{folder_destination_data}/{problem_formulation}_refset_with_variables.csv", index=False, header=True)
    else:
        nondominated = pareto.eps_sort([data.values], [32, 33, 34, 35, 36, 37, 38], [0.5,0.1,0.1,0.1,0.01,0.1,0.01], maximize=[32, 33, 34, 35, 37])
        reference_sets[problem_formulation] = pd.DataFrame(nondominated)
        reference_sets[problem_formulation] = column_renamer(reference_sets[problem_formulation])
        reference_sets[problem_formulation] = column_renamer(reference_sets[problem_formulation])
        df_nondom = pd.DataFrame(nondominated, columns=data.columns)
        df_nondom.to_csv(f"{folder_destination_data}/{problem_formulation}_refset_with_variables.csv", index=False, header=True)

Folder already exists: /Users/farleyrimon/Documents/GitHub/MUSEH2O/susquehanna/refsets


In [262]:
import pareto 

reference_sets_sol = {}

for problem_formulation in pareto_sets:
    data = pareto_sets[problem_formulation]
    if problem_formulation == 'TraditionalPrinciple':
        nondominated = pareto.eps_sort([data.values], [0,1,2,3,4,5], [0.5,0.1,0.1,0.1,0.01,0.1], maximize=[0,1,2,3,5])
        reference_sets_sol[problem_formulation] = nondominated
        df_nondom = pd.DataFrame(nondominated, columns=['hydropower', 'atomicpowerplant', 'baltimore', 'chester', 'environment', 'recreation','equity'])
        df_nondom.to_csv(f"{folder_destination_data}/{problem_formulation}_refset.csv", index=False, header=True)
    else:
        nondominated = pareto.eps_sort([data.values], [0,1,2,3,4,5,6], [0.5,0.1,0.1,0.1,0.01,0.1,0.01], maximize=[0,1,2,3,5])
        reference_sets_sol[problem_formulation] = nondominated
        df_nondom = pd.DataFrame(nondominated, columns=['hydropower', 'atomicpowerplant', 'baltimore', 'chester', 'environment', 'recreation', 'equity'])
        df_nondom.to_csv(f"{folder_destination_data}/{problem_formulation}_refset.csv", index=False, header=True)

# Logging of other objectives

## Attaching model outcomes 

We make use of the logging of objective that is implemented in the model. We extract the results using the reference sets

In [224]:
# put your destination folder here
folder_destination_input_data = '/Users/farleyrimon/Documents/GitHub/MUSEH2O/susquehanna'

for filename in os.listdir(f'{folder_destination_input_data}/data1999'):
    if filename.startswith('wAt'):
        globals()[f"{filename[:-4]}"] = np.loadtxt(f'{folder_destination_input_data}/data1999/{filename}')
    elif filename.startswith('wBa'):
        globals()[f"{filename[:-4]}"] = np.loadtxt(f'{folder_destination_input_data}/data1999/{filename}')
    elif filename.startswith('wCh'):
        globals()[f"{filename[:-4]}"] = np.loadtxt(f'{folder_destination_input_data}/data1999/{filename}')
    elif filename == 'min_flow_req.txt':
        globals()[f"{filename[:-4]}"] = np.loadtxt(os.path.join(f"{folder_destination_input_data}/data1999", filename)) 

In [225]:
import rbf_functions
from susquehanna_model import SusquehannaModel

rbf = rbf_functions.original_rbf


reference_set_traditional = reference_sets[ethical_formulations[0]]

# setup the RBF network
n_inputs = 2  # (time, storage of Conowingo)
n_outputs = 4 # Atomic, Baltimore, Chester, Downstream:- (hydropower, environmental)
n_rbfs = n_inputs+2
rbf = rbf_functions.RBF(n_rbfs, n_inputs, n_outputs, rbf_function=rbf)

# Initialize model
nobjs = 6
n_years = 1
susquehanna_river_traditional = SusquehannaModel(108.5, 505.0, 5, n_years, 
                                    rbf)  # l0, l0_MR, d0, years
# l0 = start level cono, l0_MR = start level muddy run, d0 = startday > friday = 5

susquehanna_river_traditional.set_log(True)

output_traditional = []
# iterate over solutions
for _, row in reference_set_traditional.iloc[:, 0:32].iterrows():
    output_traditional.append(susquehanna_river_traditional.evaluate(row))

In [226]:
import rbf_functions
from susquehanna_model import SusquehannaModel

rbf = rbf_functions.original_rbf

# probably does not work because of the extra equity metric

reference_set_gini_mean = reference_sets[ethical_formulations[1]]

# setup the RBF network
n_inputs = 2  # (time, storage of Conowingo)
n_outputs = 4 # Atomic, Baltimore, Chester, Downstream:- (hydropower, environmental)
n_rbfs = n_inputs+2
rbf = rbf_functions.RBF(n_rbfs, n_inputs, n_outputs, rbf_function=rbf)

# Initialize model
nobjs = 7
n_years = 1
susquehanna_river_gini_mean = SusquehannaModel(108.5, 505.0, 5, n_years, 
                                    rbf)  # l0, l0_MR, d0, years
# l0 = start level cono, l0_MR = start level muddy run, d0 = startday > friday = 5

susquehanna_river_gini_mean.set_log(True)

output_gini_mean = []
# iterate over solutions
for _, row in reference_set_gini_mean.iloc[:, 0:32].iterrows():
    output_gini_mean.append(susquehanna_river_gini_mean.evaluate(row))

In [227]:
import rbf_functions
from susquehanna_model import SusquehannaModel

rbf = rbf_functions.original_rbf

reference_set_gini_std = reference_sets[ethical_formulations[2]]

# setup the RBF network
n_inputs = 2  # (time, storage of Conowingo)
n_outputs = 4 # Atomic, Baltimore, Chester, Downstream:- (hydropower, environmental)
n_rbfs = n_inputs+2
rbf = rbf_functions.RBF(n_rbfs, n_inputs, n_outputs, rbf_function=rbf)

# Initialize model
nobjs = 7
n_years = 1
susquehanna_river_gini_std = SusquehannaModel(108.5, 505.0, 5, n_years, 
                                    rbf)  # l0, l0_MR, d0, years
# l0 = start level cono, l0_MR = start level muddy run, d0 = startday > friday = 5

susquehanna_river_gini_std.set_log(True)

output_gini_std = []
# iterate over solutions
for _, row in reference_set_gini_std.iloc[:, 0:32].iterrows():
    output_gini_std.append(susquehanna_river_gini_std.evaluate(row))




In [228]:
import rbf_functions
from susquehanna_model import SusquehannaModel

rbf = rbf_functions.original_rbf

reference_set_gini_ratio = reference_sets[ethical_formulations[3]]

# setup the RBF network
n_inputs = 2  # (time, storage of Conowingo)
n_outputs = 4 # Atomic, Baltimore, Chester, Downstream:- (hydropower, environmental)
n_rbfs = n_inputs+2
rbf = rbf_functions.RBF(n_rbfs, n_inputs, n_outputs, rbf_function=rbf)

# Initialize model
nobjs = 7
n_years = 1
susquehanna_river_gini_ratio = SusquehannaModel(108.5, 505.0, 5, n_years, 
                                    rbf)  # l0, l0_MR, d0, years
# l0 = start level cono, l0_MR = start level muddy run, d0 = startday > friday = 5

susquehanna_river_gini_ratio.set_log(True)

output_gini_ratio = []
# iterate over solutions
for _, row in reference_set_gini_ratio.iloc[:, 0:32].iterrows():
    output_gini_ratio.append(susquehanna_river_gini_ratio.evaluate(row))



In [229]:
import rbf_functions
from susquehanna_model import SusquehannaModel

rbf = rbf_functions.original_rbf

reference_set_eucli_mean = reference_sets[ethical_formulations[4]]

# setup the RBF network
n_inputs = 2  # (time, storage of Conowingo)
n_outputs = 4 # Atomic, Baltimore, Chester, Downstream:- (hydropower, environmental)
n_rbfs = n_inputs+2
rbf = rbf_functions.RBF(n_rbfs, n_inputs, n_outputs, rbf_function=rbf)

# Initialize model
nobjs = 7
n_years = 1
susquehanna_river_eucli_mean = SusquehannaModel(108.5, 505.0, 5, n_years, 
                                    rbf)  # l0, l0_MR, d0, years
# l0 = start level cono, l0_MR = start level muddy run, d0 = startday > friday = 5

susquehanna_river_eucli_mean.set_log(True)

output_eucli_mean = []
# iterate over solutions
for _, row in reference_set_eucli_mean.iloc[:, 0:32].iterrows():
    output_eucli_mean.append(susquehanna_river_eucli_mean.evaluate(row))




In [230]:
import rbf_functions
from susquehanna_model import SusquehannaModel

rbf = rbf_functions.original_rbf

reference_set_eucli_std = reference_sets[ethical_formulations[5]]

# setup the RBF network
n_inputs = 2  # (time, storage of Conowingo)
n_outputs = 4 # Atomic, Baltimore, Chester, Downstream:- (hydropower, environmental)
n_rbfs = n_inputs+2
rbf = rbf_functions.RBF(n_rbfs, n_inputs, n_outputs, rbf_function=rbf)

# Initialize model
nobjs = 7
n_years = 1
susquehanna_river_eucli_std = SusquehannaModel(108.5, 505.0, 5, n_years, 
                                    rbf)  # l0, l0_MR, d0, years
# l0 = start level cono, l0_MR = start level muddy run, d0 = startday > friday = 5

susquehanna_river_eucli_std.set_log(True)

output_eucli_std = []
# iterate over solutions
for _, row in reference_set_eucli_std.iloc[:, 0:32].iterrows():
    output_eucli_std.append(susquehanna_river_eucli_std.evaluate(row))




In [231]:
import rbf_functions
from susquehanna_model import SusquehannaModel

rbf = rbf_functions.original_rbf

reference_set_eucli_ratio = reference_sets[ethical_formulations[6]]

# setup the RBF network
n_inputs = 2  # (time, storage of Conowingo)
n_outputs = 4 # Atomic, Baltimore, Chester, Downstream:- (hydropower, environmental)
n_rbfs = n_inputs+2
rbf = rbf_functions.RBF(n_rbfs, n_inputs, n_outputs, rbf_function=rbf)

# Initialize model
nobjs = 7
n_years = 1
susquehanna_river_eucli_ratio = SusquehannaModel(108.5, 505.0, 5, n_years, 
                                    rbf)  # l0, l0_MR, d0, years
# l0 = start level cono, l0_MR = start level muddy run, d0 = startday > friday = 5

susquehanna_river_eucli_ratio.set_log(True)

output_eucli_ratio = []
# iterate over solutions
for _, row in reference_set_eucli_ratio.iloc[:, 0:32].iterrows():
    output_eucli_ratio.append(susquehanna_river_eucli_ratio.evaluate(row))




Now we can finally look at our trade-off visualization since we have the equity values for the traditional problem formulation. Previously this was not part of the solutions output, because equity is not optimized for the traditional problem formulation.

## Log model outcomes

In [232]:

columns = ['blevel_CO', 'blevel_MR', 'ratom', 'rbalt', 'rches', 'renv', 'gini_yearly_mean_coeff', 'eucli_yearly_mean_coeff', 'gini_monthly_std_coeff', 'eucli_monthly_std_coeff', 'j_hydro_reliability_yearly_mean', 'gini_monthly', 'eucli_monthly', 'gini_ratio','eucli_ratio']


traditional_log = susquehanna_river_traditional.get_log()
df_traditional_log = pd.DataFrame({columns[0]: traditional_log[0],
                                  columns[1]: traditional_log[1],
                                  columns[2]: traditional_log[2],
                                  columns[3]: traditional_log[3],
                                  columns[4]: traditional_log[4],
                                  columns[5]: traditional_log[5],
                                  columns[6]: traditional_log[6],
                                  columns[7]: traditional_log[7],
                                  columns[8]: traditional_log[8],
                                  columns[9]: traditional_log[9],
                                  columns[10]: traditional_log[10],
                                  columns[11]: traditional_log[11],
                                  columns[12]: traditional_log[12],
                                  columns[13]: traditional_log[13],
                                  columns[14]: traditional_log[14]
                                  })
df_traditional_log.to_csv(f"{folder_destination_data}/{ethical_formulations[0]}_log.csv", index=False, header=True)



gini_mean_log = susquehanna_river_gini_mean.get_log()
df_gini_mean_log = pd.DataFrame({columns[0]: gini_mean_log[0],
                                  columns[1]: gini_mean_log[1],
                                  columns[2]: gini_mean_log[2],
                                  columns[3]: gini_mean_log[3],
                                  columns[4]: gini_mean_log[4],
                                  columns[5]: gini_mean_log[5],
                                  columns[6]: gini_mean_log[6],
                                  columns[7]: gini_mean_log[7],
                                  columns[8]: gini_mean_log[8],
                                  columns[9]: gini_mean_log[9],
                                  columns[10]: gini_mean_log[10],
                                  columns[11]: gini_mean_log[11],
                                  columns[12]: gini_mean_log[12],
                                  columns[13]: gini_mean_log[13],
                                  columns[14]: gini_mean_log[14]
                                  })
df_gini_mean_log.to_csv(f"{folder_destination_data}/{ethical_formulations[1]}_log.csv", index=False, header=True)


gini_std_log = susquehanna_river_gini_std.get_log()
df_gini_std_log = pd.DataFrame({columns[0]: gini_std_log[0],
                                  columns[1]: gini_std_log[1],
                                  columns[2]: gini_std_log[2],
                                  columns[3]: gini_std_log[3],
                                  columns[4]: gini_std_log[4],
                                  columns[5]: gini_std_log[5],
                                  columns[6]: gini_std_log[6],
                                  columns[7]: gini_std_log[7],
                                  columns[8]: gini_std_log[8],
                                  columns[9]: gini_std_log[9],
                                  columns[10]: gini_std_log[10],
                                  columns[11]: gini_std_log[11],
                                  columns[12]: gini_std_log[12],
                                  columns[13]: gini_std_log[13],
                                  columns[14]: gini_std_log[14]
                                  })
df_gini_std_log.to_csv(f"{folder_destination_data}/{ethical_formulations[2]}_log.csv", index=False, header=True)


gini_ratio_log = susquehanna_river_gini_ratio.get_log()
df_gini_ratio_log = pd.DataFrame({columns[0]: gini_ratio_log[0],
                                  columns[1]: gini_ratio_log[1],
                                  columns[2]: gini_ratio_log[2],
                                  columns[3]: gini_ratio_log[3],
                                  columns[4]: gini_ratio_log[4],
                                  columns[5]: gini_ratio_log[5],
                                  columns[6]: gini_ratio_log[6],
                                  columns[7]: gini_ratio_log[7],
                                  columns[8]: gini_ratio_log[8],
                                  columns[9]: gini_ratio_log[9],
                                  columns[10]: gini_ratio_log[10],
                                  columns[11]: gini_ratio_log[11],
                                  columns[12]: gini_ratio_log[12],
                                  columns[13]: gini_ratio_log[13],
                                  columns[14]: gini_ratio_log[14]
                                  })
df_gini_ratio_log.to_csv(f"{folder_destination_data}/{ethical_formulations[3]}_log.csv", index=False, header=True)


eucli_mean_log = susquehanna_river_eucli_mean.get_log()
df_eucli_mean_log = pd.DataFrame({columns[0]: eucli_mean_log[0],
                                  columns[1]: eucli_mean_log[1],
                                  columns[2]: eucli_mean_log[2],
                                  columns[3]: eucli_mean_log[3],
                                  columns[4]: eucli_mean_log[4],
                                  columns[5]: eucli_mean_log[5],
                                  columns[6]: eucli_mean_log[6],
                                  columns[7]: eucli_mean_log[7],
                                  columns[8]: eucli_mean_log[8],
                                  columns[9]: eucli_mean_log[9],
                                  columns[10]: eucli_mean_log[10],
                                  columns[11]: eucli_mean_log[11],
                                  columns[12]: eucli_mean_log[12],
                                  columns[13]: eucli_mean_log[13],
                                  columns[14]: eucli_mean_log[14],
                                  })
df_eucli_mean_log.to_csv(f"{folder_destination_data}/{ethical_formulations[4]}_log.csv", index=False, header=True)


eucli_std_log = susquehanna_river_eucli_std.get_log()
df_eucli_std_log = pd.DataFrame({columns[0]: eucli_std_log[0],
                                  columns[1]: eucli_std_log[1],
                                  columns[2]: eucli_std_log[2],
                                  columns[3]: eucli_std_log[3],
                                  columns[4]: eucli_std_log[4],
                                  columns[5]: eucli_std_log[5],
                                  columns[6]: eucli_std_log[6],
                                  columns[7]: eucli_std_log[7],
                                  columns[8]: eucli_std_log[8],
                                  columns[9]: eucli_std_log[9],
                                  columns[10]: eucli_std_log[10],
                                  columns[11]: eucli_std_log[11],
                                  columns[12]: eucli_std_log[12],
                                  columns[13]: eucli_std_log[13],
                                  columns[14]: eucli_std_log[14]
                                  })
df_eucli_std_log.to_csv(f"{folder_destination_data}/{ethical_formulations[5]}_log.csv", index=False, header=True)


eucli_ratio_log = susquehanna_river_eucli_ratio.get_log()
df_eucli_ratio_log = pd.DataFrame({columns[0]: eucli_ratio_log[0],
                                  columns[1]: eucli_ratio_log[1],
                                  columns[2]: eucli_ratio_log[2],
                                  columns[3]: eucli_ratio_log[3],
                                  columns[4]: eucli_ratio_log[4],
                                  columns[5]: eucli_ratio_log[5],
                                  columns[6]: eucli_ratio_log[6],
                                  columns[7]: eucli_ratio_log[7],
                                  columns[8]: eucli_ratio_log[8],
                                  columns[9]: eucli_ratio_log[9],
                                  columns[10]: eucli_ratio_log[10],
                                  columns[11]: eucli_ratio_log[11],
                                  columns[12]: eucli_ratio_log[12],
                                  columns[13]: eucli_ratio_log[13],
                                  columns[14]: eucli_ratio_log[14]
                                  })
df_eucli_ratio_log.to_csv(f"{folder_destination_data}/{ethical_formulations[6]}_log.csv", index=False, header=True)


# Preprocessing the data before creating visualization plots

In [233]:
def normalizer(df):
    # Access the euclidean mean so we normalize it between the normal ranges of each coefficient.
    dataframe = df.copy()
    equity_column = dataframe['Equity']
    
    equity_values = equity_column.values.reshape(-1, 1)  # Reshape the column to a 2D array for normalization
    scaler = MinMaxScaler()
    
    # Normalize the column between 0 and 1
    normalized_equity = scaler.fit_transform(equity_values)
    
    # Update the 'Equity' column with the normalized values
    dataframe['Equity'] = normalized_equity
    
    return dataframe

def column_renamer(df):
    new_columns = ['Hydropower revenue', 'Atomic PP', 'Baltimore', 'Chester', 'Environment', 'Recreation', 'Equity']
    # Get the current column names
    current_columns = df.columns

    # Create a mapping dictionary to map old column names to new column names
    mapping = dict(zip(current_columns[-7:], new_columns))

    # Rename the columns using the mapping dictionary
    df = df.rename(columns=mapping)
    return df

In [263]:
df_traditional = pd.read_csv(f"{folder_destination_data}/{ethical_formulations[0]}_log.csv")
df_gini_mean_log = pd.read_csv(f"{folder_destination_data}/{ethical_formulations[1]}_log.csv")
df_gini_std_log = pd.read_csv(f"{folder_destination_data}/{ethical_formulations[2]}_log.csv")
df_gini_ratio_log = pd.read_csv(f"{folder_destination_data}/{ethical_formulations[3]}_log.csv")
df_eucli_mean_log = pd.read_csv(f"{folder_destination_data}/{ethical_formulations[4]}_log.csv")
df_eucli_std_log = pd.read_csv(f"{folder_destination_data}/{ethical_formulations[5]}_log.csv")
df_eucli_ratio_log = pd.read_csv(f"{folder_destination_data}/{ethical_formulations[6]}_log.csv")
df_log_list = [df_traditional_log,df_gini_mean_log,df_gini_ratio_log, df_gini_std_log,df_eucli_mean_log,df_eucli_std_log,df_eucli_ratio_log]


In [234]:
df_log_list = [df_traditional_log,df_gini_mean_log,df_gini_ratio_log, df_gini_std_log,df_eucli_mean_log,df_eucli_std_log,df_eucli_ratio_log]

reference_set_traditional = column_renamer(reference_set_traditional)
reference_set_gini_mean = column_renamer(reference_set_gini_mean)
reference_set_gini_std = column_renamer(reference_set_gini_std)
reference_set_gini_ratio = column_renamer(reference_set_gini_ratio)
reference_set_eucli_mean = column_renamer(reference_set_eucli_mean)
reference_set_eucli_mean = normalizer(reference_set_eucli_mean)
reference_set_eucli_std = column_renamer(reference_set_eucli_std)
reference_set_eucli_ratio = column_renamer(reference_set_eucli_ratio)

df_refset_list = [reference_set_traditional,reference_set_gini_mean,reference_set_gini_std,reference_set_gini_ratio, reference_set_eucli_mean,reference_set_eucli_std,reference_set_eucli_ratio]


In [235]:
# folder_destination_data = '/Users/farleyrimon/Documents/GitHub/MUSEH2O/susquehanna/refsets' 
# folder_destination = '/Users/farleyrimon/Documents/GitHub/MUSEH2O/susquehanna/refsets' 

# reference_sets = {}
# for entry in ethical_formulations:
#     reference_sets[entry] = pd.read_csv(os.path.join(f'{folder_destination_data}', f"{entry}_refset_with_variables.csv"))


In [236]:
# reference_set_traditional = reference_sets[ethical_formulations[0]]
# reference_set_gini_mean = reference_sets[ethical_formulations[1]]
# reference_set_gini_std = reference_sets[ethical_formulations[2]]
# reference_set_gini_ratio = reference_sets[ethical_formulations[3]]
# reference_set_eucli_mean = reference_sets[ethical_formulations[4]]
# reference_set_eucli_std = reference_sets[ethical_formulations[5]]
# reference_set_eucli_ratio = reference_sets[ethical_formulations[6]]

In [237]:
new_columns = ['Hydropower revenue', 'Atomic PP', 'Baltimore', 'Chester', 'Environment', 'Recreation', 'Gini Mean', 'Euclidean Mean', 'Gini Std', 'Euclidean Std', 'Gini Ratio', 'Euclidean Ratio', 'Formulation']

dict_principles_objectives = {}
dict_df_logs = {}

j=0
for i in ethical_formulations:
    dict_df_logs[i] = df_log_list[j]
    j+=1
    
j=0
for i in ethical_formulations:
    df_temp = df_refset_list[j]
    df_temp = pd.DataFrame(df_temp.iloc[:,-7:])
    
    #convert to desired preference direction for Parallel Axis Plot
    df_temp['Equity'] = 1-df_temp['Equity'] # equity
    df_temp['Environment'] = 1-df_temp['Environment'] # environment
    dict_principles_objectives[i] = df_temp
    j+=1

# dict_principles_objectives[ethical_formulations[1]] = normalizer(dict_principles_objectives[ethical_formulations[1]])
# dict_principles_objectives[ethical_formulations[2]] = normalizer(dict_principles_objectives[ethical_formulations[2]])
# dict_principles_objectives[ethical_formulations[3]] = normalizer(dict_principles_objectives[ethical_formulations[3]])
# dict_principles_objectives[ethical_formulations[4]] = normalizer(dict_principles_objectives[ethical_formulations[4]])
# dict_principles_objectives[ethical_formulations[5]] = normalizer(dict_principles_objectives[ethical_formulations[5]])
# dict_principles_objectives[ethical_formulations[6]] = normalizer(dict_principles_objectives[ethical_formulations[6]])
    
    

dictionary saved successfully to file


In [273]:
def calculate_row_std(df):
    std_values = np.std(df, axis=1)
    return std_values

In [311]:
import pickle

dict_principles_all_inequality_metrics = {}
dict_principles_reliability_values = {}

for entry in ethical_formulations:
    df_temp = dict_df_logs[entry]
    df_temp_2 = df_temp[['gini_yearly_mean_coeff', 'eucli_yearly_mean_coeff', 'gini_monthly_std_coeff', 'eucli_monthly_std_coeff','gini_ratio','eucli_ratio']]
    
    df_temp_3 = dict_principles_objectives[entry]
    df_temp_3 = pd.DataFrame(df_temp_3.iloc[:,-7:])
    df_new = pd.concat([df_temp_3, df_temp_2], axis=1)

    df_new['principle'] = entry
    df_new.to_csv(f"{folder_destination_data}/{entry}_all_inequality_metrics.csv", index=False, header=True)
    dict_principles_all_inequality_metrics[entry] = df_new
    
    df_temp_4 = df_temp[['j_hydro_reliability_yearly_mean']]
    df_new_2 = pd.concat([df_temp_3.iloc[:,-7:-1], df_temp_4], axis=1)
    df_new_2 = df_new_2.drop(['Hydropower revenue'], axis=1)

    df_new_2['Deviation'] = calculate_row_std(df_new_2)
    df_new_2 = df_new_2.drop('j_hydro_reliability_yearly_mean',axis=1)

    column_name_front = 'Hydropower revenue'  
    column_to_move_front = df_temp_3['Hydropower revenue']
    
    column_name_back = 'Equity'  
    column_to_move_back = df_temp_3['Equity']


    df_new_2.insert(0, column_name_front, column_to_move_front)
    df_new_2.insert(6, column_name_back, column_to_move_back)
    
    df_new_2['principle'] = entry
    df_new_2.to_csv(f"{folder_destination_data}/{entry}_reliability_values.csv", index=False, header=True)
    dict_principles_reliability_values[entry] = df_new_2

    
# save dictionary to .pkl file
with open(f'{folder_destination_data}/dict_principles_all_inequality_metrics.pkl', 'wb') as fp:
    pickle.dump(dict_principles_all_inequality_metrics, fp)
    print('dictionary saved successfully to file')
    
# save dictionary to .pkl file
with open(f'{folder_destination_data}/dict_principles_reliability_values.pkl', 'wb') as fp:
    pickle.dump(dict_principles_reliability_values, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file
dictionary saved successfully to file


In [313]:
dict_principles_objectives[ethical_formulations[4]]

,Hydropower revenue,Atomic PP,Baltimore,Chester,Environment,Recreation,Equity
0,76.092404,0.494702,0.139603,0.737238,0.920397,1.000000,0.067103
1,77.304088,0.966786,0.666656,0.935750,0.910396,1.000000,0.344140
2,22.263592,0.219604,0.221418,0.164742,0.834810,0.642857,0.773832
3,76.350140,0.794965,0.090499,0.778981,0.920025,1.000000,0.081390
4,33.744959,0.418919,0.389207,0.445714,0.900425,0.750000,0.662887
...,...,...,...,...,...,...,...
1117,37.256918,0.460310,0.446903,0.469892,0.677639,0.821429,0.798957
1118,43.006996,0.905992,0.586657,0.583802,0.795821,0.750000,0.546759
1119,55.058984,0.672870,0.633472,0.670760,0.849996,0.857143,0.625595
1120,46.840387,0.674492,0.560211,0.568792,0.790277,0.714286,0.750517
